In [1]:
import numpy as np
import pandas as pd
import re
from nltk.tokenize import word_tokenize 

import pickle
import collections 

In [2]:
def sanitize(text):
    text = text.decode('utf-8')
    text = text.lower()
    return ' '.join(word_tokenize(text))

In [3]:
data_path = '/home/vk352/paraphraseDomainShift/data/'
spm_path = '/home/vk352/paraphraseDomainShift/'

In [47]:
data_quora = pd.read_csv(data_path+'quora_duplicate_questions.tsv', sep='\t')
f = open("pretrainQuora.txt", "a")

for index, row in data_quora[12000:120000].iterrows():
    f.write(row['question1']+'\n')
    f.write(row['question2']+'\n')
f.close()

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [4]:
# quora data
data_quora = pd.read_csv(data_path+'quora_duplicate_questions.tsv', sep='\t')
data_quora = data_quora[['question1', 'question2', 'is_duplicate']]
data_quora.rename(columns={'is_duplicate': 'paraphrase'}, inplace=True)
data_quora.dropna(inplace=True)
# data_quora['question1'] = data_quora['question1'].apply(lambda text: sanitize(text))
# data_quora['question2'] = data_quora['question2'].apply(lambda text: sanitize(text))
# data_quora.to_pickle('data_quora')

In [5]:
data_quora.rename(columns={'is_duplicate': 'paraphrase', 'question1': 'utt1', 'question2': 'utt2'}, inplace=True)

In [17]:
data_quora.to_pickle(data_path+'data_quora')

In [33]:
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/quora/train/a.toks', train['utt1'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/quora/train/b.toks', train['utt2'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/quora/train/sim.txt', train['paraphrase'].values, fmt='%s')

np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/quora/test/a.toks', test['utt1'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/quora/test/b.toks', test['utt2'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/quora/test/sim.txt', test['paraphrase'].values, fmt='%s')

np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/quora/dev/a.toks', dev['utt1'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/quora/dev/b.toks', dev['utt2'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/quora/dev/sim.txt', dev['paraphrase'].values, fmt='%s')

In [9]:
data_quora = pd.read_pickle(data_path+'data_quora')
test = data_quora[-10000:]
dev = data_quora[:-10000][-10000:]
train = data_quora[:-20000]

In [6]:
# msr_data 
with open(data_path+'msr-paraphrase-corpus/msr_paraphrase_train.txt', 'r') as file:
    lines = file.readlines() 
data_msr = pd.DataFrame(data={'utt1': [line.split('\t')[3] for line in lines[1:]],
                             'utt2': [line.split('\t')[4] for line in lines[1:]],
                             'paraphrase': [line.split('\t')[0] for line in lines[1:]]})
with open(data_path+'msr-paraphrase-corpus/msr_paraphrase_test.txt', 'r') as file:
    lines = file.readlines() 
data_msr_test = pd.DataFrame(data={'utt1': [line.split('\t')[3] for line in lines[1:]],
                             'utt2': [line.split('\t')[4] for line in lines[1:]],
                             'paraphrase': [line.split('\t')[0] for line in lines[1:]]})

In [49]:
data_msr['utt1'] = data_msr['utt1'].apply(lambda text: sanitize(text))
data_msr['utt2'] = data_msr['utt2'].apply(lambda text: sanitize(text))
data_msr_test['utt1'] = data_msr_test['utt1'].apply(lambda text: sanitize(text))
data_msr_test['utt2'] = data_msr_test['utt2'].apply(lambda text: sanitize(text))
split = int(0.8*len(data_msr))
data_msr_train = data_msr[:split]
data_msr_val = data_msr[split:]

# np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/msr/train/a.toks', data_msr_train['utt1'].values, fmt='%s')
# np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/msr/train/b.toks', data_msr_train['utt2'].values, fmt='%s')
# np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/msr/train/sim.txt', data_msr_train['paraphrase'].values, fmt='%s')

# np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/msr/test/a.toks', data_msr_test['utt1'].values, fmt='%s')
# np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/msr/test/b.toks', data_msr_test['utt2'].values, fmt='%s')
# np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/msr/test/sim.txt', data_msr_test['paraphrase'].values, fmt='%s')

# np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/msr/dev/a.toks', data_msr_val['utt1'].values, fmt='%s')
# np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/msr/dev/b.toks', data_msr_val['utt2'].values, fmt='%s')
# np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/msr/dev/sim.txt', data_msr_val['paraphrase'].values, fmt='%s')

In [11]:
li = []
li_set = set()
for df in [data_quora, data_msr, data_msr_test, data_twitter, data_twitter_test, data_paws_train, data_paws_test, data_paws_dev,
           data_paws_qqp_train, data_paws_qqp_test, data_paws_qqp_val]:
    li = []
    for sent in list(df['utt1'])+list(df['utt2']):
        stext = sanitize(sent)
        li.extend(stext.split(' '))
    a = set(li)
    li_set.update(a)
    print(len(a))


d = {}
for i, elem in enumerate(li_set):
    d[elem] = i
# di = collections.OrderedDict(sorted(d.items(), key=itemgetter(1)))
# pickle.dump( di, open( "msr_vocab_cased.pkl", "wb" ) )

113435
14800
9288
31344
13342
31511
9072
9164
18178
2038
1974


In [12]:
pickle.dump( d, open( "vocab.pkl", "wb" ) )

In [99]:
a = pickle.load( open( "/home/vk352/paraphraseDomainShift/SPM_toolkit/DecAtt/data/vocab.pkl", "rb" ) )

In [103]:
len(list(a.keys()))

136420

In [350]:
# d = {'banana': 3, 'apple':4, 'pear': 1, 'orange': 2}
sorted_dict = collections.OrderedDict(sorted(d.items(), key=lambda t: t[1]))

In [353]:
len(sorted_dict), len(li)

(13008, 13008)

In [7]:
#twitter data
data_twitter = pd.read_csv(data_path+'twitter/Twitter_URL_Corpus_train.txt', sep='\t', header=None)
data_twitter.columns = ['utt1', 'utt2', 'paraphrase', 'link']
data_twitter = data_twitter[['utt1', 'utt2', 'paraphrase']]
data_twitter['paraphrase'] = data_twitter['paraphrase'].apply(lambda x: int(re.search('(.),', x).group(1)))
# original paper removes entries where annotators are split
data_twitter = data_twitter[data_twitter['paraphrase']!=3] 
data_twitter['paraphrase'] = data_twitter['paraphrase'].apply(lambda x: '1' if x>=4 else '0')

data_twitter_test = pd.read_csv(data_path+'twitter/Twitter_URL_Corpus_test.txt', sep='\t', header=None)
data_twitter_test.columns = ['utt1', 'utt2', 'paraphrase', 'link']
data_twitter_test = data_twitter_test[['utt1', 'utt2', 'paraphrase']]
data_twitter_test['paraphrase'] = data_twitter_test['paraphrase'].apply(lambda x: int(re.search('(.),', x).group(1)))
# original paper removes entries where annotators are split
data_twitter_test = data_twitter_test[data_twitter_test['paraphrase']!=3] 
data_twitter_test['paraphrase'] = data_twitter_test['paraphrase'].apply(lambda x: '1' if x>=4 else '0')

In [16]:
data_twitter['utt1'] = data_twitter['utt1'].apply(lambda text: sanitize(text))
data_twitter['utt2'] = data_twitter['utt2'].apply(lambda text: sanitize(text))
data_twitter_test['utt1'] = data_twitter_test['utt1'].apply(lambda text: sanitize(text))
data_twitter_test['utt2'] = data_twitter_test['utt2'].apply(lambda text: sanitize(text))
split = int(0.8*len(data_twitter))
data_twitter_train = data_twitter[:split]
data_twitter_val = data_twitter[split:]

np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/twitter/train/a.toks', data_twitter_train['utt1'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/twitter/train/b.toks', data_twitter_train['utt2'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/twitter/train/sim.txt', data_twitter_train['paraphrase'].values, fmt='%s')

np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/twitter/test/a.toks', data_twitter_test['utt1'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/twitter/test/b.toks', data_twitter_test['utt2'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/twitter/test/sim.txt', data_twitter_test['paraphrase'].values, fmt='%s')

np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/twitter/dev/a.toks', data_twitter_val['utt1'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/twitter/dev/b.toks', data_twitter_val['utt2'].values, fmt='%s')
np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/twitter/dev/sim.txt', data_twitter_val['paraphrase'].values, fmt='%s')

In [8]:
# Paws qqp
data_paws_qqp_train = pd.read_csv(data_path+'PAWS/paws_qqp/train.tsv', sep='\t', skiprows=1, names=['id', 'utt1', 'utt2', 'paraphrase'], encoding="utf-8")
data_paws_qqp_dev_test = pd.read_csv(data_path+'PAWS/paws_qqp/dev_and_test.tsv', sep='\t', skiprows=1, names=['id', 'utt1', 'utt2', 'paraphrase'])

# remove b''
data_paws_qqp_train['utt1'] = data_paws_qqp_train.apply(lambda row: row['utt1'][2:-1],axis=1)
data_paws_qqp_train['utt2'] = data_paws_qqp_train.apply(lambda row: row['utt2'][2:-1],axis=1)
data_paws_qqp_dev_test['utt1'] = data_paws_qqp_dev_test.apply(lambda row: row['utt1'][2:-1],axis=1)
data_paws_qqp_dev_test['utt2'] = data_paws_qqp_dev_test.apply(lambda row: row['utt2'][2:-1],axis=1)

data_paws_qqp_val = data_paws_qqp_dev_test[:int(len(data_paws_qqp_dev_test)/2)]
data_paws_qqp_test = data_paws_qqp_dev_test[int(len(data_paws_qqp_dev_test)/2):]
data_paws_qqp_val = data_paws_qqp_val.reset_index()
data_paws_qqp_test = data_paws_qqp_test.reset_index()

In [26]:
for df, name in zip([data_paws_qqp_train, data_paws_qqp_test, data_paws_qqp_val], ['train', 'test', 'dev']):
    df['utt1'] = df['utt1'].apply(lambda text: sanitize(text))
    df['utt2'] = df['utt2'].apply(lambda text: sanitize(text))
    
    np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/paws_qqp/%s/a.toks'%format(name), df['utt1'].values, fmt='%s')
    np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/paws_qqp/%s/b.toks'%format(name), df['utt2'].values, fmt='%s')
    np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/paws_qqp/%s/sim.txt'%format(name), df['paraphrase'].values, fmt='%s')

In [9]:
# paws
data_paws_train = pd.read_csv(data_path+'PAWS/final/train.tsv', sep='\t', skiprows=1, names=['id', 'utt1', 'utt2', 'paraphrase'])
data_paws_test = pd.read_csv(data_path+'PAWS/final/test.tsv', sep='\t', skiprows=1, names=['id', 'utt1', 'utt2', 'paraphrase'])
data_paws_dev = pd.read_csv(data_path+'PAWS/final/dev.tsv', sep='\t', skiprows=1, names=['id', 'utt1', 'utt2', 'paraphrase'])


In [29]:
for df, name in zip([data_paws_train, data_paws_test, data_paws_dev], ['train', 'test', 'dev']):
    df['utt1'] = df['utt1'].apply(lambda text: sanitize(text))
    df['utt2'] = df['utt2'].apply(lambda text: sanitize(text))
    
    np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/paws/%s/a.toks'%format(name), df['utt1'].values, fmt='%s')
    np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/paws/%s/b.toks'%format(name), df['utt2'].values, fmt='%s')
    np.savetxt(spm_path+'SPM_toolkit/DecAtt/data/paws/%s/sim.txt'%format(name), df['paraphrase'].values, fmt='%s')

In [13]:
len(data_paws_train), len(data_paws_dev), len(data_paws_test)

(49401, 8000, 8000)

In [34]:
tokens = pickle.load( open( "/home/vk352/paraphraseDomainShift/SPM_toolkit/DecAtt/data/vocab.pkl", "rb" ) )
tokens = list(tokens.keys())

In [47]:
'mass.' in tokens

False

In [19]:
b = []
s = 'which one is correct? “please let me know if you have any question” or ”please let me know if you have any questions”'
sanitize(s)

u'which one is correct ? \u201c please let me know if you have any question \u201d or \u201d please let me know if you have any questions \u201d'

In [57]:
data_msr['utt2'].apply(lambda text: sanitize(text)).iloc[2224]

u'in july , the hopkinton , mass. , company agreed to buy legato systems of mountain view for $ 1.3 billion .'

In [62]:
data_msr['utt2'].iloc[2224]

'In July, the Hopkinton, Mass., company agreed to buy Legato Systems of Mountain View for $1.3 billion.\n'